In [0]:
# reference
# https://towardsdatascience.com/getting-census-data-in-5-easy-steps-a08eeb63995d
# data list
# https://api.census.gov/data/2018/acs/acs5/variables.html

In [1]:
#manipulate dataframes in python
import pandas as pd

#make API calls with python
import requests

#allows us to store results of API call cleanly
import json

In [2]:
#display all the columns in dataframe
pd.set_option('display.max_columns', None)

In [3]:
#put your census API key here
apiKey = "09e7e4a144645fc4e247d25e6477d1a245f561cd"

#Put variable code
var_code = 'DP05_0018E'

#construct the API call we will use
baseAPI = "https://api.census.gov/data/2018/acs/acs5/profile?get=%s&for=tract:*&in=state:27&in=county:053&key=%s" 
# 
calledAPI = baseAPI % (var_code, apiKey)
print(calledAPI)

#call the API and collect the response
response = requests.get(calledAPI)

#load the response into a JSON, ignoring the first element which is just field labels
formattedResponse = json.loads(response.text)


https://api.census.gov/data/2018/acs/acs5/profile?get=DP05_0018E&for=tract:*&in=state:27&in=county:053&key=09e7e4a144645fc4e247d25e6477d1a245f561cd


In [8]:
#store the response in a dataframe, change the first column name to the variable name you want to use in database
data = pd.DataFrame(columns=[ 'Median Age', 'state', 'county', 'tract'], data=formattedResponse)

#Align the column formats
data[['state', 'county', 'tract']] = data[['state', 'county', 'tract']].astype(str)

#Join the TractId column
data['TractId'] = data[['state', 'county', 'tract']].agg(''.join, axis=1)

In [7]:
data.head()

,Median Age,state,county,tract,TractId
0,DP05_0018E,state,county,tract,statecountytract
1,28.0,27,053,103700,27053103700
2,24.6,27,053,104000,27053104000
3,30.4,27,053,104100,27053104100
4,20.1,27,053,104900,27053104900


In [9]:
# Select columns we need
data_by_Tract = data.iloc[1: ,[0,4]]
data_by_Tract  = data_by_Tract [reversed(data_by_Tract .columns)]

In [10]:
data_by_Tract.head()

,TractId,Median Age
1,27053103700,28.0
2,27053104000,24.6
3,27053104100,30.4
4,27053104900,20.1
5,27053001100,33.2


In [329]:
# Create a dataframe to store all attributes at the same place, do this if you are pulling data for the first time
All_data_by_Tract = data_by_Tract

In [516]:
# Run this code to append the attributes to existing dataframe, do this if you already have the tract level dataframe
All_data_by_Tract = pd.merge(All_data_by_Tract, data_by_Tract, on='TractId')

In [518]:
#save that dataframe to a CSV spreadsheet
All_data_by_Tract.to_csv('All_data_by_Tract.csv', index=False)